In [1]:
import sys
sys.path.append("../../")
from STK_Sim import *

Filename = 'Communications_SetUp'

stk_object = STK_Simulation(False,Filename)

In [2]:
# Comparing high data rate w/ no interpolation to low data rate w/ interpolation data

stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "UTCG")
start_time = time_convert(stk_object.root.CurrentScenario.StartTime)
duration = datetime.timedelta(days=1, seconds=0, microseconds=0, milliseconds=0, minutes=0, hours=0, weeks=0)
stop_time=(start_time+duration).strftime("%d %b %Y %H:%M:%S.%f")
stk_object.root.CurrentScenario.StopTime=stop_time
stk_object.root.UnitPreferences.SetCurrentUnit("DateFormat", "EpSec")

# Loading targets into stk from file.
# targets = stk_object.Target_Loader("../../Input_Files/Targets_File.txt")
sats = stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")

## Facility

In [3]:
# IAgStkObjectRoot root: STK Object Model Root
facility = stk_object.root.CurrentScenario.Children.New(AgESTKObjectType.eFacility, 'USHI02') # eFacility

# IAgFacility facility: Facility Object
facility.Position.AssignGeodetic(19.0138,155.6629,0.0) # Latitude, Longitude, Altitude

# Set altitude to height of terrain
facility.UseTerrain = True

# Set altitude to a distance above the ground
facility.HeightAboveGround = 0.92  # km

#### Transmission

In [4]:
transmitter = facility.Children.New(AgESTKObjectType.eTransmitter, 'GSTransmitter')
transmitter.SetModel('Complex Transmitter Model')
txModel = transmitter.Model
txModel.SetModulator('BPSK')
txModel.Frequency = 7.19 # GHz
txModel.Power = 34.77 # dBW
txModel.DataRate = 56 # Mb/sec

In [5]:
antennaControltx = txModel.AntennaControl
antennaControltx.SetEmbeddedModel('Parabolic')
antennaControltx.EmbeddedModel.DesignFrequency = 7.19 #GHz
antennaControltx.EmbeddedModel.InputType = 0
antennaControltx.EmbeddedModel.Beamwidth = 0.19 # deg
antennaControltx.EmbeddedModel.Efficiency = 100

In [6]:
txModel.EnablePolarization = True
txModel.SetPolarizationType(2)
# txModel.Polarization
#dir(txModel)

In [7]:
accessConstraints = transmitter.AccessConstraints

In [8]:
constraintArray = accessConstraints.AvailableConstraints()
print('List of Available Constraints:')
for i in range(0,len(constraintArray)):
   print(constraintArray[i])

List of Available Constraints:
['3DTilesMask', 221]
['Altitude', 2]
['AngleOffBoresight', 75]
['AngularRate', 3]
['ApparentTime', 4]
['AtmosLoss', 148]
['AzElMask', 68]
['AzimuthAngle', 6]
['AzimuthRate', 69]
['BER+I', 149]
['BitErrorRate', 150]
['C/I', 151]
['C/N', 152]
['C/N+I', 153]
['C/No', 154]
['C/No+Io', 155]
['CbObstruction', 91]
['CentralAngle', 222]
['CentralDistance', 223]
['CloudsFogLoss', 156]
['CrdnAngle', 9]
['CrdnCondition', 104]
['CrdnVectorMag', 10]
['DeltaT/T', 158]
['DopplerShift', 159]
['Duration', 13]
['Eb/No', 160]
['Eb/No+Io', 161]
['ElevationAngle', 14]
['ElevationRate', 70]
['FluxDensity', 162]
['FreeSpaceLoss', 177]
['Frequency', 163]
['G/T', 164]
['GeoExclusion', 71]
['GMT', 16]
['GroundSampleDistance', 72]
['HeightAboveHorizon', 73]
['Intervals', 22]
['IonoFadingLoss', 0]
['J/S', 165]
['Lighting', 25]
['LineOfSight', 26]
['LinkEIRP', 166]
['LinkMargin', 167]
['LocalTime', 27]
['LOSLunarExclusion', 28]
['LOSSunExclusion', 29]
['LOSSunIlluminationAngle', 187]

In [9]:
linkEIRP = accessConstraints.AddNamedConstraint('LinkEIRP')
linkEIRP.EnableMax = True
linkEIRP.Max = 86

In [10]:
gain = txModel.PostTransmitGainsLosses.Add(57.8) # dBi
gain.Identifier = 'Transmitting Gain'

#### Receiving

In [11]:
receiver = facility.Children.New(AgESTKObjectType.eReceiver, 'GSReceiver')
receiver.SetModel('Complex Receiver Model')
recModel = receiver.Model
recModel.AutoTrackFrequency = 8.35 # GHz

In [12]:
recModel.LinkMargin.Enable = True
recModel.LinkMargin.Type = 1
recModel.LinkMargin.Threshold = 10.5 # dB

In [13]:
antennaControlrx = recModel.AntennaControl
antennaControlrx.SetEmbeddedModel('Parabolic')

In [14]:
antennaControlrx.EmbeddedModel.DesignFrequency = 8.35 #GHz
antennaControlrx.EmbeddedModel.InputType = 0
antennaControlrx.EmbeddedModel.Beamwidth = 0.17 # deg
antennaControlrx.EmbeddedModel.Efficiency = 100

In [15]:
gain = recModel.PreReceiveGainsLosses.Add(59.1) # dBi
gain.Identifier = 'Receiving Gain'

In [16]:
recModel.EnablePolarization = True
recModel.SetPolarizationType(2)
# recModel.Polarization

In [17]:
accessConstraints2 = receiver.AccessConstraints

In [18]:
systemSensitivity = accessConstraints2.AddNamedConstraint('G/T')
systemSensitivity.EnableMax = True
systemSensitivity.Max = 37.7 # db/K

In [19]:
recModel.SystemNoiseTemperature.ConstantNoiseTemperature = 220 # K  

In [20]:
facilities = [facility]

## Coverage

In [25]:
stk_object.Satellite_Loader("../../Input_Files/Satellites_File.txt")
stk_object.dt = 3600
stk_object.set_sim_time(weeks=1)
uplink = stk_object.Get_Access_DP(facilities, stk_object.satellites, "AER Data/Default", ['Time','Elevation'])

- Computing_AER Data/Default [==========] 10/10 [100%] in 0.2s (42.37/s)        


In [26]:
downlink = stk_object.Get_Access_DP(stk_object.satellites, facilities, "AER Data/Default", ['Time','Elevation'])

- Computing_AER Data/Default [==========] 10/10 [100%] in 0.2s (43.39/s)        


In [23]:
LLA_State = stk_object.Get_Satellite_DP("LLA State/Fixed")

- Computing_LLA State/Fixed [==========] 10/10 [100%] in 3.8s (2.50/s)          


In [24]:
LLA_State

[         time        lat           lon         alt     alt agl  lat rate  \
 0         0.0   0.008555 -4.516278e-14  711.000000  710.982839  0.060959   
 1      3600.0 -38.342240  1.640647e+02  719.190736  719.175437 -0.060670   
 2      7200.0  76.390220 -3.475749e+01  731.191969  728.769845  0.060068   
 3     10800.0 -65.573598 -4.263265e+01  728.704433  728.691512  0.060296   
 4     14400.0  27.428911  1.204227e+02  715.510228  715.033024 -0.060800   
 ..        ...        ...           ...         ...         ...       ...   
 188  676800.0 -23.308247  5.276673e+01  714.326271  714.328745  0.060842   
 189  680400.0 -15.070511 -1.430660e+02  712.435320  712.440175 -0.060909   
 190  684000.0  53.311985  2.067950e+01  724.713418  724.545829  0.060467   
 191  687600.0 -88.256076  4.756366e+01  732.364728  729.412077  0.045862   
 192  691200.0  50.645514  1.734943e+02  723.746150  723.746045 -0.060504   
 
      lon rate  alt rate  centric lat   centric lon  terrain alt     alt m